# Script to study MCTrue neutrino interactions in C++

Here, some very basic information covered:

* What does the tree contain (tracks, MCPoints);

* Some common used functions

First, we define a function to get particle charge from its MonteCarlo PdgCode: https://pdg.lbl.gov/2007/reviews/montecarlorpp.pdf

In [1]:
double GetParticleCharge (int pdgcode, TDatabasePDG *pdg){
  //from PDG, get charge
  double charge = 0.;
  if (pdg->GetParticle(pdgcode)) charge = pdg->GetParticle(pdgcode)->Charge();
  else if (pdgcode > 1e+8) charge = 1.; //these are heavy nuclei, for now we give them charge 1.Add a decode function with info from (https://pdg.lbl.gov/2007/reviews/montecarlorpp.pdf)
  return charge;
}

## Importing the simulation file

We import the simulation files. We usually produce many small simulation files in parallel, and we merge them in a chain

The simulation file is usually called **cbmsim** 

In [2]:
TChain treechain("cbmsim");
//newest simulation files from Cristovao Vilela
treechain.Add("/eos/user/a/aiulian/sim_snd/cristovao_simulation_sndlhc_13TeV_down_volTarget_SNDG18_02a_01_000/128/sndLHC.Genie-TGeant4.root");
treechain.Add("/eos/user/a/aiulian/sim_snd/cristovao_simulation_sndlhc_13TeV_down_volTarget_SNDG18_02a_01_000/206/sndLHC.Genie-TGeant4.root");
treechain.Add("/eos/user/a/aiulian/sim_snd/cristovao_simulation_sndlhc_13TeV_down_volTarget_SNDG18_02a_01_000/209/sndLHC.Genie-TGeant4.root");
treechain.Add("/eos/user/a/aiulian/sim_snd/cristovao_simulation_sndlhc_13TeV_down_volTarget_SNDG18_02a_01_000/211/sndLHC.Genie-TGeant4.root");
treechain.Add("/eos/user/a/aiulian/sim_snd/cristovao_simulation_sndlhc_13TeV_down_volTarget_SNDG18_02a_01_000/312/sndLHC.Genie-TGeant4.root");
TTreeReader reader(&treechain);
TTreeReaderArray<ShipMCTrack> tracks(reader,"MCTrack");
TTreeReaderArray<ScifiPoint> emudetpoints(reader,"ScifiPoint");

In [3]:
//variables to be used in the main code
 int ievent; //event number in tree
 double weight; //statistical weight, related to density and length of tranversed material
 //primary neutrino info
 int nu_pdg, dau_pdg;
 double nu_vx, nu_vy, nu_vz, nu_p, dau_p, dau_charge;

In [4]:
const int nentries = reader.GetEntries(true);
cout<<"Number of events "<<nentries<<endl;

Number of events 4034


## Preparing histograms

We prepare here some simple ROOT histograms, for position and momentum.

We will consider neutrinos, and charged neutrino daughters

In [5]:
TH1D *hnu_p = new TH1D("hnu_p","Neutrino momentum;P[GeV/c]",50,0,5000);
TH2D *hnu_vxy = new TH2D("hnu_vxy","vxy of all considered interactions;vx[cm];vy[cm]",120,-60,0,120,0,60);
TH1D *hnu_vz = new TH1D("hnu_vz","vz of all considered interactions;vz[cm]",150,250,400);

TH1D *hdau_p = new TH1D("hdau_p","Neutrino daughter momentum;P[GeV/c]",50,0,5000);

## Main Loop

We are ready for our loop.

We loop over the events, and over the tracks from each event

In [6]:
TDatabasePDG *pdgdatabase = TDatabasePDG::Instance();
for(int ientry = 0;ientry<nentries;ientry++){
   reader.SetEntry(ientry);
   //first track is the neutrino. Nota Bene: it "starts" at the neutrino interaction vertex, where it actually ends! 
   //The neutrino is the only "special case", since we did not simulate in Geant4 neutrino propagation
   nu_vx = tracks[0].GetStartX();
   nu_vy = tracks[0].GetStartY();
   nu_vz = tracks[0].GetStartZ();
   nu_p = tracks[0].GetP();
   //filling neutrino histograms 
   hnu_vxy->Fill(nu_vx,nu_vy);
   hnu_vz->Fill(nu_vz);
   hnu_p->Fill(nu_p);
   //loop over tracks
   for (const ShipMCTrack& track: tracks){     
         //look for charged particles from primary vertex
         if (track.GetMotherId()==0){ //daughter of neutrino, increasing counter at the end
          dau_p = track.GetP();

          dau_pdg = track.GetPdgCode();
          dau_charge = GetParticleCharge(dau_pdg, pdgdatabase);

          if (TMath::Abs(dau_charge) > 0) hdau_p->Fill(dau_p);

         }
   }
}

## Drawing the histograms

We are ready to draw the histograms. 

Calling **%jsroot on** allows the plots to be interactive

In [7]:
%jsroot on

In [8]:
TCanvas *cnupos = new TCanvas();
cnupos->Divide(1,2);
cnupos->cd(1);
hnu_vxy->Draw("COLZ");
cnupos->cd(2);
hnu_vz->Draw();
cnupos->Draw();

In [9]:
TCanvas *cp = new TCanvas();
hdau_p->Draw("histo");
hdau_p->SetLineColor(kRed);
hnu_p->Draw("histo&&SAMES");
cp->Draw();